In [1]:
from pathlib import Path

import pandas as pd
import numpy as np
import nibabel as nib
from nilearn.image import resample_img, new_img_like
from templateflow.api import get
import arviz as az
import bambi as bmb
import pytensor
import numpyro


pd.set_option("display.max_rows", 500)
numpyro.set_host_device_count(8)


def get_data_paths(suffix: str):
    if suffix in {"mask"}:
        pass
    elif suffix in {"z"}:
        suffix = f"stat-{suffix}_statmap"

    for subject in range(1, 17):
        yield Path("../data/ENIGMA") / (
            f"sub-{subject:02d}_task-faces_feature-taskBased_"
            "taskcontrast-facesGtScrambled_"
            "model-aggregateTaskBasedAcrossRuns_"
            f"contrast-intercept_{suffix}.nii.gz"
        )

In [2]:
template_path = get(
    template="MNI152NLin2009cAsym", resolution=2, desc="brain", suffix="T1w"
)
template_image = nib.load(template_path)

template_mask_path = get(
    template="MNI152NLin2009cAsym", resolution=2, desc="brain", suffix="mask"
)
template_mask_image = nib.load(template_mask_path)
template_mask_data = np.asanyarray(template_mask_image.dataobj, dtype=bool)

In [3]:
target_affine = template_image.affine
target_affine[:3, :3] *= 2.5
target_affine

array([[   5. ,    0. ,    0. ,  -96.5],
       [   0. ,    5. ,    0. , -132.5],
       [   0. ,    0. ,    5. ,  -78.5],
       [   0. ,    0. ,    0. ,    1. ]])

In [4]:
mask = np.all(
    np.concatenate(
        [
            np.asanyarray(
                resample_img(
                    nib.Nifti1Image.from_filename(mask_path),
                    target_affine,
                    interpolation="nearest",
                ).dataobj
            ).astype(bool)[:, :, :, np.newaxis]
            for mask_path in get_data_paths("mask")
        ],
        axis=3,
    ),
    axis=3,
)

/var/folders/xq/sl7xjp_95qx7wfngy6s6vvg00000gn/T/ipykernel_44059/1255261835.py:5: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  resample_img(
/var/folders/xq/sl7xjp_95qx7wfngy6s6vvg00000gn/T/ipykernel_44059/1255261835.py:5: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  resample_img(
/var/folders/xq/sl7xjp_95qx7wfngy6s6vvg00000gn/T/ipykernel_44059/1255261835.py:5: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  resample_img(
/var/folders/xq/sl7xjp_95qx7wfngy6s6vvg00000gn/T/ipykernel_44059/1255261835.py:5: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header

In [5]:
x, y, z = np.nonzero(mask)
len(x)

14752

In [6]:
data_array = np.concatenate(
    [
        resample_img(
            nib.Nifti1Image.from_filename(zstat_path),
            target_affine,
        ).get_fdata()[x, y, z, np.newaxis]
        for zstat_path in get_data_paths("z")
    ],
    axis=1,
)

/var/folders/xq/sl7xjp_95qx7wfngy6s6vvg00000gn/T/ipykernel_44059/2491353669.py:3: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  resample_img(
/var/folders/xq/sl7xjp_95qx7wfngy6s6vvg00000gn/T/ipykernel_44059/2491353669.py:3: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  resample_img(
/var/folders/xq/sl7xjp_95qx7wfngy6s6vvg00000gn/T/ipykernel_44059/2491353669.py:3: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  resample_img(
/var/folders/xq/sl7xjp_95qx7wfngy6s6vvg00000gn/T/ipykernel_44059/2491353669.py:3: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header

In [7]:
data_frame = pd.DataFrame(data_array)
data_frame["voxel"] = np.ravel_multi_index((x, y, z), mask.shape)

data_frame = data_frame.melt(id_vars=["voxel"], var_name="subject")

In [8]:
data_frame

,voxel,subject,value
0,10095,0,-0.071804
1,10096,0,0.112519
2,10097,0,1.102691
3,10098,0,0.886668
4,10099,0,0.028903
...,...,...,...
236027,63060,15,0.801796
236028,63061,15,0.560480
236029,63062,15,0.135751
236030,63099,15,-0.280793


In [9]:
# Convert the 'value' column from float64 to float32
# filtered_data_frame.loc[:, "value"] = filtered_data_frame["value"].astype("float32")
# import pytensor
#pytensor.config.floatX = "float32"
model = bmb.Model("value ~ (1|subject) + (1|voxel)", data_frame)

In [10]:
results = model.fit(
    tune=1000,
    draws=1000,
    chains=4,
    inference_method="nutpie",
    nuts_kwargs=dict(max_tree_depth=100),
)

Progress,Draws,Divergences,Step Size,Gradients/Draw
,1300,0,0.14,31
,1300,0,0.15,31
,1300,0,0.14,31
,1300,0,0.16,31
,1300,0,0.14,31
,1300,0,0.14,31
,1300,0,0.15,31
,1300,0,0.14,31


In [11]:
results.to_netcdf("results.nc")

'results.nc'

In [12]:
number_of_divergences = int(results.sample_stats.diverging.sum())
number_of_divergences

0

In [13]:
bool(np.all((0.9 <= az.rhat(results) <= 1.05).to_array()))

False

In [14]:
effect = results.posterior["1|voxel"]

In [15]:
x, y, z = np.unravel_index(list(map(int, effect.voxel__factor_dim)), mask.shape)

In [16]:
posterior_map = np.zeros(template_image.shape)
posterior_map[x, y, z] = effect.mean(axis=(0, 1))

In [17]:
posterior_map_image = resample_img(
    nib.Nifti1Image(posterior_map, target_affine),
    template_image.affine,
    interpolation="nearest",
)

/var/folders/xq/sl7xjp_95qx7wfngy6s6vvg00000gn/T/ipykernel_44059/1027666291.py:1: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  posterior_map_image = resample_img(
/var/folders/xq/sl7xjp_95qx7wfngy6s6vvg00000gn/T/ipykernel_44059/1027666291.py:1: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  posterior_map_image = resample_img(


In [18]:
nib.save(posterior_map_image, "posterior.nii.gz")

In [19]:
probability_map = np.zeros(template_image.shape)
probability_map[x, y, z] = ((effect > 0).mean(axis=(0, 1)) - 0.5) * 2

probability_map_image = resample_img(
    nib.Nifti1Image(probability_map, target_affine),
    template_image.affine,
    interpolation="nearest",
)

nib.save(probability_map_image, "probability.nii.gz")

/var/folders/xq/sl7xjp_95qx7wfngy6s6vvg00000gn/T/ipykernel_44059/1671385454.py:4: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  probability_map_image = resample_img(
/var/folders/xq/sl7xjp_95qx7wfngy6s6vvg00000gn/T/ipykernel_44059/1671385454.py:4: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  probability_map_image = resample_img(
